In [1]:
import pandas as pd
import numpy as np
import os
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
import pickle
from sklearn.metrics import plot_confusion_matrix
from builtins import range
from builtins import object
from resizeimage import resizeimage
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
CLASS_NAMES = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
CLASS_NAMES_WITHOUT_DISGUST = ["Angry", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
FILE_NAME = "face_augmentated_3.csv"
WHITE_IMAGES = [6458,  7629, 10423, 11286, 13148, 13402, 13988, 15894, 22198, 22927, 28601, 59]
IMAGES_FOR_TESTING = "data/Images_for_testing/"

In [3]:
data_path = ["data/"] # Insert data file path
file_path = os.sep.join(data_path + [FILE_NAME])
data = pd.read_csv(file_path)
data = data.drop(index=WHITE_IMAGES, axis=0)
data_exclude_disgust = data[data['emotion'] != 1] # Drops the emotion Disgust

In [4]:
def data_X_y(data):
    # Split data into X & y
    X = data.drop('emotion', axis='columns')
    y = data['emotion']

    # Reshapes X into 3D array
    X = [pixels.split(" ") for pixels in data["pixels"]]
    X = np.array(X)
    X = X.astype("int32")
    X = np.array([image.reshape(48, 48) for image in X])
    X = X/255.0
    X = X.reshape(len(X), 48, 48, 1)
    
    return X,y

In [5]:
def reshape_X(X):
    num_training = X.shape[0]
    mask = list(range(num_training))
    X_reshape = X[mask]

    # Reshape the image data into rows
    X_reshape = np.reshape(X, (X.shape[0], -1))
    
    return X_reshape

In [6]:
X,y = data_X_y(data_exclude_disgust)
X = reshape_X(X)


In [7]:
rosParams = [['hinge', 'l2', True, True, False, True], ['hinge', 'l2', False, True, False, True], ['hinge', 'elasticnet', True, True, False, True], ['hinge', 'elasticnet', False, True, False, True], ['log', 'l2', True, True, False, True], ['log', 'l2', False, True, False, False], ['log', 'l1', True, False, False, True], ['log', 'l1', True, False, False, False], ['log', 'elasticnet', True, True, False, True], ['log', 'elasticnet', True, True, False, False], ['log', 'elasticnet', False, True, False, True], ['modified_huber', 'l1', True, True, False, True]]
smoteParams = [['hinge', 'l2', True, True, False, False], ['hinge', 'l2', True, False, False, True], ['hinge', 'l1', True, True, False, True], ['hinge', 'elasticnet', True, True, False, True], ['hinge', 'elasticnet', True, True, False, False], ['hinge', 'elasticnet', True, False, False, True], ['hinge', 'elasticnet', True, False, False, False], ['hinge', 'elasticnet', False, True, False, True], ['hinge', 'elasticnet', False, True, False, False], ['hinge', 'elasticnet', False, False, False, True], ['log', 'l2', True, False, False, True], ['log', 'l2', True, False, False, False], ['log', 'l2', False, True, False, False], ['log', 'l1', True, True, False, True], ['log', 'l1', True, True, False, False], ['log', 'l1', True, False, False, True], ['log', 'l1', True, False, False, False], ['log', 'l1', False, True, False, True], ['log', 'elasticnet', True, False, False, True], ['log', 'elasticnet', True, False, False, False], ['log', 'elasticnet', False, True, False, True], ['log', 'elasticnet', False, True, False, False], ['modified_huber', 'l2', True, True, False, True], ['modified_huber', 'elasticnet', False, True, False, False], ['squared_hinge', 'l1', True, True, False, True]]

In [9]:
rosResults = []
counter = 0
filename = 'rosParams'

for i in rosParams:               
    clf = SGDClassifier(loss = rosParams[counter][0], penalty =  rosParams[counter][1], fit_intercept = rosParams[counter][2], shuffle = rosParams[counter][3], early_stopping = rosParams[counter][4], warm_start = rosParams[counter][5])
    print("Current params: ", clf)
    counter = counter + 1

    clf.fit(X, y)
    
    filename = filename + str(i)
    pickle.dump(clf, open(filename, 'wb'))
    print("Model saved to: ",filename)

Current params:  SGDClassifier(warm_start=True)
Model saved to:  rosParams['hinge', 'l2', True, True, False, True]
Current params:  SGDClassifier(fit_intercept=False, warm_start=True)
Model saved to:  rosParams['hinge', 'l2', True, True, False, True]['hinge', 'l2', False, True, False, True]
Current params:  SGDClassifier(penalty='elasticnet', warm_start=True)
Model saved to:  rosParams['hinge', 'l2', True, True, False, True]['hinge', 'l2', False, True, False, True]['hinge', 'elasticnet', True, True, False, True]
Current params:  SGDClassifier(fit_intercept=False, penalty='elasticnet', warm_start=True)
Model saved to:  rosParams['hinge', 'l2', True, True, False, True]['hinge', 'l2', False, True, False, True]['hinge', 'elasticnet', True, True, False, True]['hinge', 'elasticnet', False, True, False, True]
Current params:  SGDClassifier(loss='log', warm_start=True)
Model saved to:  rosParams['hinge', 'l2', True, True, False, True]['hinge', 'l2', False, True, False, True]['hinge', 'elasticn

OSError: [Errno 22] Invalid argument: "rosParams['hinge', 'l2', True, True, False, True]['hinge', 'l2', False, True, False, True]['hinge', 'elasticnet', True, True, False, True]['hinge', 'elasticnet', False, True, False, True]['log', 'l2', True, True, False, True]['log', 'l2', False, True, False, False]"

In [ ]:
smoteResults = []
counter = 0
filename = 'smoteParams'

for i in smoteParams:               
    clf = SGDClassifier(loss = smoteParams[counter][0], penalty =  smoteParams[counter][1], fit_intercept = smoteParams[counter][2], shuffle = smoteParams[counter][3], early_stopping = smoteParams[counter][4], warm_start = smoteParams[counter][5])
    print("Current params: ", clf)
    counter = counter + 1

    clf.fit(X, y)

    filename = filename + str(i)
    
    pickle.dump(clf, open(filename, 'wb'))
    print("Model saved to:",filename)